# Build Simple State Model with AsyncMachineBuilder

## 1. State Diagram

```lua
stateDiagram-v2
    INIT --> GENERATE
    GENERATE --> FINAL
```

```mermaid
stateDiagram-v2
    INIT --> GENERATE
    GENERATE --> FINAL
```

---

## 2. Use AsyncMachineBuilder


In [ ]:
from gai.asm import AsyncStateMachine

async def generate_action(state):
    
    from openai import AsyncOpenAI
    client = AsyncOpenAI()
    
    # Import data from state_bag
    agent_name = state.machine.state_bag.get("name", "Assistant")
    user_message = state.machine.state_bag.get("user_message", "If you are seeing this, that means I have forgotten to add a user message. Remind me.")
    monologue = state.input["monologue"]
    
    from openai import AsyncOpenAI
    client = AsyncOpenAI()
    
    # Execute
    
    monologue.add_user_message(state="INIT",content=f"Your name is {agent_name}. You are a helpful assistant.{agent_name}, {user_message}")
    response = await client.chat.completions.create(
        model="gpt-4.1",
        messages=monologue.list_chat_messages(),
        max_tokens=50,
        stream=True
    )
    
    async def streamer():
        content = ""
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                content += chunk
                yield chunk
        monologue.add_assistant_message(state="GENERATE", content=content)

    state.machine.state_bag["streamer"] = streamer()

## Step 1: INIT

with AsyncStateMachine.StateMachineBuilder("""
    INIT --> GENERATE
    GENERATE --> FINAL
    """
    ) as builder:
    fsm = builder.build({
        "INIT": {
            "input_data": {
                "name": "Sara",
                "user_message": "Hello, world!",
                "dialogue_messages": []
            },
        },
        "GENERATE": {
            "module_path": "gai.asm.states",
            "class_name": "PureActionState",
            "title": "GENERATE",
            "action": "generate",
            "output_data": ["streamer"]            
        },
        "FINAL": {
            "output_data": ["monologue"],
        }
    },generate=generate_action)

## Step 2: INIT --> GENERATE

await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk, end='', flush=True)
print("\n\n")

## Step 3: GENERATE --> FINAL
await fsm.run_async()

## Step 4: Print the state history
print("State History:")
for state in fsm.state_history:
    print(f"State: {state['state']}")
    print(f"- input: {state['input']}")
    print(f"- output: {state['output']}")
    print("-" * 20)

AttributeError: 'StateModel' object has no attribute 'name'